# New York City Taxi Demand Prediction

# -----------------------------------------------------------------------------------------------------------

# Data Information

<p>
Ge the data from : http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml (2016 data)
The data used in the attached datasets were collected and provided to the NYC Taxi and Limousine Commission (TLC) 
</p>

## Information on taxis:

<h5> Yellow Taxi: Yellow Medallion Taxicabs</h5>
<p> These are the famous NYC yellow taxis that provide transportation exclusively through street-hails. The number of taxicabs is limited by a finite number of medallions issued by the TLC. You access this mode of transportation by standing in the street and hailing an available taxi with your hand. The pickups are not pre-arranged.</p>

<h5> For Hire Vehicles (FHVs) </h5>
<p> FHV transportation is accessed by a pre-arrangement with a dispatcher or limo company. These FHVs are not permitted to pick up passengers via street hails, as those rides are not considered pre-arranged. </p>

<h5> Green Taxi: Street Hail Livery (SHL) </h5>
<p>  The SHL program will allow livery vehicle owners to license and outfit their vehicles with green borough taxi branding, meters, credit card machines, and ultimately the right to accept street hails in addition to pre-arranged rides. </p>
<p> Credits: Quora</p>

<h5>Footnote:</h5>
In the given notebook we are considering only the yellow taxis for the time period between Jan - Mar 2015 & Jan - Mar 2016

# ML Problem Formulation
<p><b> Time-series forecasting and Regression</b></p>
<br>
-<i> To find number of pickups, given location cordinates(latitude and longitude) and time, in the query reigion and surrounding regions.</i>
<p> 
To solve the above we would be using data collected in Jan - Mar 2015 to predict the pickups in Jan - Mar 2016.
</p>

# Performance metrics
1. Mean Absolute percentage error.

# -----------------------------------------------------------------------------------------------------------

# Objectives:
    1.Apply Regression Models with Fourier features
    2.Hyper Parameter Tuning For Regression Models
    3.Feature Engineering To Bring MAPE less than 0.12

# Findings :

### Task 1: Regression Models With Foureir Features
     
     XGB Mape
        Train:0.1279.Test:0.1257
        
     Random Forest Mape
        Train:0.0935.Test:0.1246
     
     Linear Regression Mape
        Train:0.1327.Test:0.1286

### Task 2: Regression Models With HyperParam Tunning
     
     XGB Mape
        Train:0.1203.Test:0.1238
        
     Random Forest Mape
        Train:0.1175.Test:0.1245
     
     Linear Regression Mape
        Train:0.1330.Test:0.1292

### Task 3: Feature Engineering
     
     Random Forest Mape
        Train:0.1124.Test:0.1193

# -----------------------------------------------------------------------------------------------------------

In [1]:
import dask.dataframe as dd#similar to pandas
import pandas as pd#pandas to create small dataframes 
import folium #open street map
import datetime #Convert to unix time
import time #Convert to unix time
import numpy as np#Do aritmetic operations on arrays

import matplotlib
# matplotlib.use('nbagg') : matplotlib uses this protocall which makes plots more user intractive like zoom in and zoom out
matplotlib.use('nbagg')
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  

import gpxpy.geo #Get the haversine distance

from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor

import math
import pickle
import os
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Looking at the features
# dask dataframe  : # https://github.com/dask/dask-tutorial/blob/master/07_dataframe.ipynb
month = dd.read_csv('yellow_tripdata_2015-01.csv')
print(month.columns)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')


# -----------------------------------------------------------------------------------------------------------

## Data Cleaning


<h3> Based on analysis done on 2015 Jan data we have the following findings </h3>
<ol>
    <li> Trim outliers for lat and long lying outside NY boundaries </li>
    <li> Trim outliers for trip time between  0 and 720</li>
    <li> Trim outliers for trip distance between 0 and 23 </li>
    <li> Trim outliers for speed between 0 and 65 </li>
    <li> Trim outliers for fare between 0 and 1000  </li>
</ol>


# -----------------------------------------------------------------------------------------------------------

<h5>Function to convert to Unix time</h5>

In [3]:
def convert_to_unix(s):
    return time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple())

def return_with_trip_times(month):
    duration = month[['tpep_pickup_datetime','tpep_dropoff_datetime']].compute()
    #pickups and dropoffs to unix time
    duration_pickup = [convert_to_unix(x) for x in duration['tpep_pickup_datetime'].values]
    duration_drop = [convert_to_unix(x) for x in duration['tpep_dropoff_datetime'].values]
    #calculate duration of trips
    durations = (np.array(duration_drop) - np.array(duration_pickup))/float(60)

    #append durations of trips and speed in miles/hr to a new dataframe
    new_frame = month[['passenger_count','trip_distance','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','total_amount']].compute()
    
    new_frame['trip_times'] = durations
    new_frame['pickup_times'] = duration_pickup
    new_frame['Speed'] = 60*(new_frame['trip_distance']/new_frame['trip_times'])
    
    return new_frame
frame_with_durations = return_with_trip_times(month)

<h5>Function to Remove Outliers </h5>

In [10]:


#removing all outliers based on our univariate analysis above
def remove_outliers(new_frame):

    
    a = new_frame.shape[0]
    print ("Number of pickup records = ",a)
    temp_frame = new_frame[((new_frame.dropoff_longitude >= -74.15) & (new_frame.dropoff_longitude <= -73.7004) &\
                       (new_frame.dropoff_latitude >= 40.5774) & (new_frame.dropoff_latitude <= 40.9176)) & \
                       ((new_frame.pickup_longitude >= -74.15) & (new_frame.pickup_latitude >= 40.5774)& \
                       (new_frame.pickup_longitude <= -73.7004) & (new_frame.pickup_latitude <= 40.9176))]
    b = temp_frame.shape[0]
    print ("Number of outlier coordinates lying outside NY boundaries:",(a-b))

    
    temp_frame = new_frame[(new_frame.trip_times > 0) & (new_frame.trip_times < 720)]
    c = temp_frame.shape[0]
    print ("Number of outliers from trip times analysis:",(a-c))
    
    
    temp_frame = new_frame[(new_frame.trip_distance > 0) & (new_frame.trip_distance < 23)]
    d = temp_frame.shape[0]
    print ("Number of outliers from trip distance analysis:",(a-d))
    
    temp_frame = new_frame[(new_frame.Speed <= 65) & (new_frame.Speed >= 0)]
    e = temp_frame.shape[0]
    print ("Number of outliers from speed analysis:",(a-e))
    
    temp_frame = new_frame[(new_frame.total_amount <1000) & (new_frame.total_amount >0)]
    f = temp_frame.shape[0]
    print ("Number of outliers from fare analysis:",(a-f))
    
    
    new_frame = new_frame[((new_frame.dropoff_longitude >= -74.15) & (new_frame.dropoff_longitude <= -73.7004) &\
                       (new_frame.dropoff_latitude >= 40.5774) & (new_frame.dropoff_latitude <= 40.9176)) & \
                       ((new_frame.pickup_longitude >= -74.15) & (new_frame.pickup_latitude >= 40.5774)& \
                       (new_frame.pickup_longitude <= -73.7004) & (new_frame.pickup_latitude <= 40.9176))]
    
    new_frame = new_frame[(new_frame.trip_times > 0) & (new_frame.trip_times < 720)]
    new_frame = new_frame[(new_frame.trip_distance > 0) & (new_frame.trip_distance < 23)]
    new_frame = new_frame[(new_frame.Speed < 45.31) & (new_frame.Speed > 0)]
    new_frame = new_frame[(new_frame.total_amount <1000) & (new_frame.total_amount >0)]
    
    print ("Total outliers removed",a - new_frame.shape[0])
    print ("---")
    return new_frame

# -----------------------------------------------------------------------------------------------------------

# Data-preperation
## Clustering/Segmentation

### Inference:
- The main objective was to find a optimal min. distance(Which roughly estimates to the radius of a cluster) between the clusters which we got was 40

In [12]:
#trying different cluster sizes to choose the right K in K-means
coords = frame_with_durations_outliers_removed[['pickup_latitude', 'pickup_longitude']].values
neighbours=[]

def find_min_distance(cluster_centers, cluster_len):
    nice_points = 0
    wrong_points = 0
    less2 = []
    more2 = []
    min_dist=1000
    for i in range(0, cluster_len):
        nice_points = 0
        wrong_points = 0
        for j in range(0, cluster_len):
            if j!=i:
                distance = gpxpy.geo.haversine_distance(cluster_centers[i][0], cluster_centers[i][1],cluster_centers[j][0], cluster_centers[j][1])
                min_dist = min(min_dist,distance/(1.60934*1000))
                if (distance/(1.60934*1000)) <= 2:
                    nice_points +=1
                else:
                    wrong_points += 1
        less2.append(nice_points)
        more2.append(wrong_points)
    neighbours.append(less2)
    print ("On choosing a cluster size of ",cluster_len,"\nAvg. Number of Clusters within the vicinity (i.e. intercluster-distance < 2):", np.ceil(sum(less2)/len(less2)), "\nAvg. Number of Clusters outside the vicinity (i.e. intercluster-distance > 2):", np.ceil(sum(more2)/len(more2)),"\nMin inter-cluster distance = ",min_dist,"\n---")

def find_clusters(increment):
    kmeans = MiniBatchKMeans(n_clusters=increment, batch_size=10000,random_state=42).fit(coords)
    frame_with_durations_outliers_removed['pickup_cluster'] = kmeans.predict(frame_with_durations_outliers_removed[['pickup_latitude', 'pickup_longitude']])
    cluster_centers = kmeans.cluster_centers_
    cluster_len = len(cluster_centers)
    return cluster_centers, cluster_len

# we need to choose number of clusters so that, there are more number of cluster regions 
#that are close to any cluster center
# and make sure that the minimum inter cluster should not be very less
for increment in range(10, 100, 10):
    cluster_centers, cluster_len = find_clusters(increment)
    find_min_distance(cluster_centers, cluster_len)            

On choosing a cluster size of  10 
Avg. Number of Clusters within the vicinity (i.e. intercluster-distance < 2): 2.0 
Avg. Number of Clusters outside the vicinity (i.e. intercluster-distance > 2): 8.0 
Min inter-cluster distance =  1.0945442325142543 
---
On choosing a cluster size of  20 
Avg. Number of Clusters within the vicinity (i.e. intercluster-distance < 2): 4.0 
Avg. Number of Clusters outside the vicinity (i.e. intercluster-distance > 2): 16.0 
Min inter-cluster distance =  0.7131298007387813 
---
On choosing a cluster size of  30 
Avg. Number of Clusters within the vicinity (i.e. intercluster-distance < 2): 8.0 
Avg. Number of Clusters outside the vicinity (i.e. intercluster-distance > 2): 22.0 
Min inter-cluster distance =  0.5185088176172206 
---
On choosing a cluster size of  40 
Avg. Number of Clusters within the vicinity (i.e. intercluster-distance < 2): 8.0 
Avg. Number of Clusters outside the vicinity (i.e. intercluster-distance > 2): 32.0 
Min inter-cluster distance 

In [13]:
# if check for the 50 clusters you can observe that there are two clusters with only 0.3 miles apart from each other
# so we choose 40 clusters for solve the further problem

# Getting 40 clusters using the kmeans 
kmeans = MiniBatchKMeans(n_clusters=40, batch_size=10000,random_state=0).fit(coords)
frame_with_durations_outliers_removed['pickup_cluster'] = kmeans.predict(frame_with_durations_outliers_removed[['pickup_latitude', 'pickup_longitude']])

# -----------------------------------------------------------------------------------------------------------

<h1> Time-binning </h1>

<h3> We make 10 min time bins as we had discovered in EDA that the average trip time based on distance is 10 min </h3>

<h5>Function to convert to 10 min time bins</h5>

In [15]:
def add_pickup_bins(frame,month,year):
    unix_pickup_times=[i for i in frame['pickup_times'].values]
    unix_times = [[1420070400,1422748800,1425168000,1427846400,1430438400,1433116800],\
                    [1451606400,1454284800,1456790400,1459468800,1462060800,1464739200]]
    
    start_pickup_unix=unix_times[year-2015][month-1]
    # https://www.timeanddate.com/time/zones/est
    # (int((i-start_pickup_unix)/600)+33) : our unix time is in gmt to we are converting it to est
    tenminutewise_binned_unix_pickup_times=[(int((i-start_pickup_unix)/600)+33) for i in unix_pickup_times]
    frame['pickup_bins'] = np.array(tenminutewise_binned_unix_pickup_times)
    return frame

# -----------------------------------------------------------------------------------------------------------

# Data preparation for months of 2016 Jan Feb Mar

In [16]:
def datapreparation(month,kmeans,month_no,year_no):
    
    print ("Return with trip times..")

    frame_with_durations = return_with_trip_times(month)
    
    print ("Remove outliers..")
    frame_with_durations_outliers_removed = remove_outliers(frame_with_durations)
    
    print ("Estimating clusters..")
    frame_with_durations_outliers_removed['pickup_cluster'] = kmeans.predict(frame_with_durations_outliers_removed[['pickup_latitude', 'pickup_longitude']])
    #frame_with_durations_outliers_removed_2016['pickup_cluster'] = kmeans.predict(frame_with_durations_outliers_removed_2016[['pickup_latitude', 'pickup_longitude']])

    print ("Final groupbying..")
    final_updated_frame = add_pickup_bins(frame_with_durations_outliers_removed,month_no,year_no)
    final_groupby_frame = final_updated_frame[['pickup_cluster','pickup_bins','trip_distance']].groupby(['pickup_cluster','pickup_bins']).count()
    
    return final_updated_frame,final_groupby_frame
    
month_jan_2016 = dd.read_csv('yellow_tripdata_2016-01.csv')
month_feb_2016 = dd.read_csv('yellow_tripdata_2016-02.csv')
month_mar_2016 = dd.read_csv('yellow_tripdata_2016-03.csv')

jan_2016_frame,jan_2016_groupby = datapreparation(month_jan_2016,kmeans,1,2016)
feb_2016_frame,feb_2016_groupby = datapreparation(month_feb_2016,kmeans,2,2016)
mar_2016_frame,mar_2016_groupby = datapreparation(month_mar_2016,kmeans,3,2016)

Return with trip times..
Remove outliers..
Number of pickup records =  10906858
Number of outlier coordinates lying outside NY boundaries: 214677
Number of outliers from trip times analysis: 27190
Number of outliers from trip distance analysis: 79742
Number of outliers from speed analysis: 21047
Number of outliers from fare analysis: 4991
Total outliers removed 297784
---
Estimating clusters..
Final groupbying..
Return with trip times..
Remove outliers..
Number of pickup records =  11382049
Number of outlier coordinates lying outside NY boundaries: 223161
Number of outliers from trip times analysis: 27670
Number of outliers from trip distance analysis: 81902
Number of outliers from speed analysis: 22437
Number of outliers from fare analysis: 5476
Total outliers removed 308177
---
Estimating clusters..
Final groupbying..
Return with trip times..
Remove outliers..
Number of pickup records =  12210952
Number of outlier coordinates lying outside NY boundaries: 232444
Number of outliers fro

# -----------------------------------------------------------------------------------------------------------

# Smoothing And Filling

<h3> We did smoothing for 2015 data </h3>
<h3> We will use filling for 2016 data so as to avoid data leakage </h3>
<h3> Removed smoothing code as it is not required here </h3>

In [17]:
# Gets the unique bins where pickup values are present for each each reigion

# for each cluster region we will collect all the indices of 10min intravels in which the pickups are happened
# we got an observation that there are some pickpbins that doesnt have any pickups
def return_unq_pickup_bins(frame):
    values = []
    for i in range(0,40):
        new = frame[frame['pickup_cluster'] == i]
        list_unq = list(set(new['pickup_bins']))
        list_unq.sort()
        values.append(list_unq)
    return values

In [23]:
# for every month we get all indices of 10min intravels in which atleast one pickup got happened

#jan
jan_2016_unique = return_unq_pickup_bins(jan_2016_frame)

#feb
feb_2016_unique = return_unq_pickup_bins(feb_2016_frame)

#march
mar_2016_unique = return_unq_pickup_bins(mar_2016_frame)

In [22]:
def fill_missing_1(count_values,values,bins):
    smoothed_regions=[]
    ind=0
    for r in range(0,40):
        smoothed_bins=[]
        for i in range(bins):
            if i in values[r]:
                smoothed_bins.append(count_values[ind])
                ind+=1
            else:
                smoothed_bins.append(0)
        smoothed_regions.extend(smoothed_bins)
    return smoothed_regions

In [24]:
jan_2016_smooth = fill_missing_1(jan_2016_groupby['trip_distance'].values,jan_2016_unique,4464)
feb_2016_smooth = fill_missing_1(feb_2016_groupby['trip_distance'].values,feb_2016_unique,4176)
mar_2016_smooth = fill_missing_1(mar_2016_groupby['trip_distance'].values,mar_2016_unique,4464)

In [25]:
print(len(jan_2016_smooth))
print(len(feb_2016_smooth))
print(len(mar_2016_smooth))

178560
167040
178560


In [26]:
# Making list of all the values of pickup data in every bin for a period of 3 months and storing them region-wise 
regions_cum = []

# a =[1,2,3]
# b = [2,3,4]
# a+b = [1, 2, 3, 2, 3, 4]

# number of 10min indices for jan 2015= 24*31*60/10 = 4464
# number of 10min indices for jan 2016 = 24*31*60/10 = 4464
# number of 10min indices for feb 2016 = 24*29*60/10 = 4176
# number of 10min indices for march 2016 = 24*31*60/10 = 4464
# regions_cum: it will contain 40 lists, each list will contain 4464+4176+4464 values which represents the number of pickups 
# that are happened for three months in 2016 data

for i in range(0,40):
    regions_cum.append(jan_2016_smooth[4464*i:4464*(i+1)]+feb_2016_smooth[4176*i:4176*(i+1)]+mar_2016_smooth[4464*i:4464*(i+1)])

# print(len(regions_cum))
# 40
# print(len(regions_cum[0]))
# 13104

In [27]:
print(len(regions_cum[0]))

13104


# -----------------------------------------------------------------------------------------------------------

# Time series and Fourier Transforms 

<h3> Calculation top 5 Amp and Freq </h3>

In [27]:
def applyFft(month_data, noOfBins,m):
    amp=[]
    frequency=[]
    for i in range(0,40):
        if(m=='jan'):
            amp_cluster    = np.abs( np.fft.fft(np.array(month_data)[ (noOfBins * i ) +5: noOfBins* (i+1) ]) )
            freq_cluster = np.fft.fftfreq(noOfBins, 1)
        else:
            amp_cluster    = np.abs( np.fft.fft(np.array(month_data)[ (noOfBins *i ): (noOfBins* (i+1)) ]) )
            freq_cluster = np.fft.fftfreq(noOfBins, 1)
        #Get top 5 amp and its frequencies
        top_indices=np.array(np.argsort(amp_cluster)[::-1])[0:5]
        #amp_cluster = [amp_cluster[top_indices]] * noOfBins
        #freq_cluster = [freq_cluster[top_indices]] * noOfBins
        
        amp_cluster = amp_cluster[top_indices]
        freq_cluster = freq_cluster[top_indices]
        
        amp.append(amp_cluster)
        frequency.append(freq_cluster)
    
    return amp,frequency

In [28]:
jan_2016_amp,jan_2016_freq= applyFft(jan_2016_smooth,4464,'jan')
feb_2016_amp,feb_2016_freq= applyFft(feb_2016_smooth,4176,'feb')
mar_2016_amp,mar_2016_freq= applyFft(mar_2016_smooth,4464,'mar')

In [29]:
len(jan_2016_smooth[5:])

178555

In [30]:
print(len(jan_2016_amp[0]))
print(len(jan_2016_amp))

5
40


<h3> amp1_allTimeBins has the top amplitutde for cluster and months. Same for others also </h3>

In [31]:
amp1_allTimeBins=[]
amp2_allTimeBins=[]
amp3_allTimeBins=[]
amp4_allTimeBins=[]
amp5_allTimeBins=[]
freq1_allTimeBins=[]
freq2_allTimeBins=[]
freq3_allTimeBins=[]
freq4_allTimeBins=[]
freq5_allTimeBins=[]

for i in range(0,40):
    amp1_allTimeBins.append( [jan_2016_amp[i][0]] * 4459 + [feb_2016_amp[i][0]] * 4176 + [mar_2016_amp[i][0]] * 4464 )
    amp2_allTimeBins.append( [jan_2016_amp[i][1]] * 4459 + [feb_2016_amp[i][1]] * 4176 + [mar_2016_amp[i][1]] * 4464 )
    amp3_allTimeBins.append( [jan_2016_amp[i][2]] * 4459 + [feb_2016_amp[i][2]] * 4176 + [mar_2016_amp[i][2]] * 4464 )
    amp4_allTimeBins.append( [jan_2016_amp[i][3]] * 4459 + [feb_2016_amp[i][3]] * 4176 + [mar_2016_amp[i][3]] * 4464 )
    amp5_allTimeBins.append( [jan_2016_amp[i][4]] * 4459 + [feb_2016_amp[i][4]] * 4176 + [mar_2016_amp[i][4]] * 4464 )
    
    freq1_allTimeBins.append( [jan_2016_freq[i][0]] * 4459 + [feb_2016_freq[i][0]] * 4176 + [mar_2016_freq[i][0]] * 4464 )
    freq2_allTimeBins.append( [jan_2016_freq[i][1]] * 4459 + [feb_2016_freq[i][1]] * 4176 + [mar_2016_freq[i][1]] * 4464 )
    freq3_allTimeBins.append( [jan_2016_freq[i][2]] * 4459 + [feb_2016_freq[i][2]] * 4176 + [mar_2016_freq[i][2]] * 4464 )
    freq4_allTimeBins.append( [jan_2016_freq[i][3]] * 4459 + [feb_2016_freq[i][3]] * 4176 + [mar_2016_freq[i][3]] * 4464 )
    freq5_allTimeBins.append( [jan_2016_freq[i][4]] * 4459 + [feb_2016_freq[i][4]] * 4176 + [mar_2016_freq[i][4]] * 4464 )

# -----------------------------------------------------------------------------------------------------------

# Train-Test Split
Before we start predictions using the tree based regression models we take 3 months of 2016 pickup data and split it such that for every region we have 70% data in train and 30% in test, ordered date-wise for every region

In [32]:
# Preparing data to be split into train and test, The below prepares data in cumulative form which will be later split into test and train
# number of 10min indices for jan 2015= 24*31*60/10 = 4464
# number of 10min indices for jan 2016 = 24*31*60/10 = 4464
# number of 10min indices for feb 2016 = 24*29*60/10 = 4176
# number of 10min indices for march 2016 = 24*31*60/10 = 4464
# regions_cum: it will contain 40 lists, each list will contain 4464+4176+4464 values which represents the number of pickups 
# that are happened for three months in 2016 data

# print(len(regions_cum))
# 40
# print(len(regions_cum[0]))
# 12960

# we take number of pickups that are happened in last 5 10min intravels
number_of_time_stamps = 5

# output varaible
# it is list of lists
# it will contain number of pickups 13099 for each cluster
output = []


# tsne_lat will contain 13104-5=13099 times lattitude of cluster center for every cluster
# Ex: [[cent_lat 13099times],[cent_lat 13099times], [cent_lat 13099times].... 40 lists]
# it is list of lists
tsne_lat = []


# tsne_lon will contain 13104-5=13099 times logitude of cluster center for every cluster
# Ex: [[cent_long 13099times],[cent_long 13099times], [cent_long 13099times].... 40 lists]
# it is list of lists
tsne_lon = []

# we will code each day 
# sunday = 0, monday=1, tue = 2, wed=3, thur=4, fri=5,sat=6
# for every cluster we will be adding 13099 values, each value represent to which day of the week that pickup bin belongs to
# it is list of lists
tsne_weekday = []

# its an numbpy array, of shape (523960, 5)
# each row corresponds to an entry in out data
# for the first row we will have [f0,f1,f2,f3,f4] fi=number of pickups happened in i+1th 10min intravel(bin)
# the second row will have [f1,f2,f3,f4,f5]
# the third row will have [f2,f3,f4,f5,f6]
# and so on...
tsne_feature = []


tsne_feature = [0]*number_of_time_stamps
for i in range(0,40):
    tsne_lat.append([kmeans.cluster_centers_[i][0]]*13099)
    tsne_lon.append([kmeans.cluster_centers_[i][1]]*13099)
    # jan 1st 2016 is thursday, so we start our day from 4: "(int(k/144))%7+4"
    # our prediction start from 5th 10min intravel since we need to have number of pickups that are happened in last 5 pickup bins
    tsne_weekday.append([int(((int(k/144))%7+4)%7) for k in range(5,4464+4176+4464)])
    # regions_cum is a list of lists [[x1,x2,x3..x13104], [x1,x2,x3..x13104], [x1,x2,x3..x13104], [x1,x2,x3..x13104], [x1,x2,x3..x13104], .. 40 lsits]
    tsne_feature = np.vstack((tsne_feature, [regions_cum[i][r:r+number_of_time_stamps] for r in range(0,len(regions_cum[i])-number_of_time_stamps)]))
    output.append(regions_cum[i][5:])
tsne_feature = tsne_feature[1:]

In [33]:
tsne_feature.shape
#tsne_feature[0]

(523960, 5)

In [34]:
len(tsne_lat[0])*len(tsne_lat) == tsne_feature.shape[0] == len(tsne_weekday)*len(tsne_weekday[0]) == 40*13099 == len(output)*len(output[0])

True

In [35]:
# Getting the predictions of exponential moving averages to be used as a feature in cumulative form

# upto now we computed 8 features for every data point that starts from 50th min of the day
# 1. cluster center lattitude
# 2. cluster center longitude
# 3. day of the week 
# 4. f_t_1: number of pickups that are happened previous t-1th 10min intravel
# 5. f_t_2: number of pickups that are happened previous t-2th 10min intravel
# 6. f_t_3: number of pickups that are happened previous t-3th 10min intravel
# 7. f_t_4: number of pickups that are happened previous t-4th 10min intravel
# 8. f_t_5: number of pickups that are happened previous t-5th 10min intravel

# from the baseline models we said the exponential weighted moving avarage gives us the best error
# we will try to add the same exponential weighted moving avarage at t as a feature to our data
# exponential weighted moving avarage => p'(t) = alpha*p'(t-1) + (1-alpha)*P(t-1) 
alpha=0.1

# it is a temporary array that store exponential weighted moving avarage for each 10min intravel, 
# for each cluster it will get reset
# for every cluster it contains 13104 values
predicted_values=[]

# it is similar like tsne_lat
# it is list of lists
# predict_list is a list of lists [[x5,x6,x7..x13104], [x5,x6,x7..x13104], [x5,x6,x7..x13104], [x5,x6,x7..x13104], [x5,x6,x7..x13104], .. 40 lsits]
predict_list = []
tsne_flat_exp_avg = []
for r in range(0,40):
    for i in range(0,13104):
        if i==0:
            predicted_value= regions_cum[r][0]
            predicted_values.append(0)
            continue
        predicted_values.append(predicted_value)
        predicted_value =int((alpha*predicted_value) + (1-alpha)*(regions_cum[r][i]))
    predict_list.append(predicted_values[5:])
    predicted_values=[]

In [36]:
# train, test split : 70% 30% split
# Before we start predictions using the tree based regression models we take 3 months of 2016 pickup data 
# and split it such that for every region we have 70% data in train and 30% in test,
# ordered date-wise for every region
print("size of train data :", int(13099*0.7))
print("size of test data :", int(13099*0.3))

size of train data : 9169
size of test data : 3929


In [37]:
# extracting first 9169 timestamp values i.e 70% of 13099 (total timestamps) for our training data
train_features =  [tsne_feature[i*13099:(13099*i+9169)] for i in range(0,40)]
# temp = [0]*(12955 - 9068)
test_features = [tsne_feature[(13099*(i))+9169:13099*(i+1)] for i in range(0,40)]

In [38]:
print("Number of data clusters",len(train_features), "Number of data points in trian data", len(train_features[0]), "Each data point contains", len(train_features[0][0]),"features")
print("Number of data clusters",len(train_features), "Number of data points in test data", len(test_features[0]), "Each data point contains", len(test_features[0][0]),"features")

Number of data clusters 40 Number of data points in trian data 9169 Each data point contains 5 features
Number of data clusters 40 Number of data points in test data 3930 Each data point contains 5 features


In [39]:
# extracting first 9169 timestamp values i.e 70% of 13099 (total timestamps) for our training data
tsne_train_flat_lat = [i[:9169] for i in tsne_lat]
tsne_train_flat_lon = [i[:9169] for i in tsne_lon]
tsne_train_flat_weekday = [i[:9169] for i in tsne_weekday]
tsne_train_flat_output = [i[:9169] for i in output]
tsne_train_flat_exp_avg = [i[:9169] for i in predict_list]

In [40]:
# extracting the rest of the timestamp values i.e 30% of 12956 (total timestamps) for our test data
tsne_test_flat_lat = [i[9169:] for i in tsne_lat]
tsne_test_flat_lon = [i[9169:] for i in tsne_lon]
tsne_test_flat_weekday = [i[9169:] for i in tsne_weekday]
tsne_test_flat_output = [i[9169:] for i in output]
tsne_test_flat_exp_avg = [i[9169:] for i in predict_list]

In [41]:
# the above contains values in the form of list of lists (i.e. list of values of each region), here we make all of them in one list
train_new_features = []
for i in range(0,40):
    train_new_features.extend(train_features[i])
test_new_features = []
for i in range(0,40):
    test_new_features.extend(test_features[i])

In [42]:
# converting lists of lists into sinle list i.e flatten
# a  = [[1,2,3,4],[4,6,7,8]]
# print(sum(a,[]))
# [1, 2, 3, 4, 4, 6, 7, 8]

tsne_train_lat = sum(tsne_train_flat_lat, [])
tsne_train_lon = sum(tsne_train_flat_lon, [])
tsne_train_weekday = sum(tsne_train_flat_weekday, [])
tsne_train_output = sum(tsne_train_flat_output, [])
tsne_train_exp_avg = sum(tsne_train_flat_exp_avg,[])

In [43]:
# converting lists of lists into sinle list i.e flatten
# a  = [[1,2,3,4],[4,6,7,8]]
# print(sum(a,[]))
# [1, 2, 3, 4, 4, 6, 7, 8]

tsne_test_lat = sum(tsne_test_flat_lat, [])
tsne_test_lon = sum(tsne_test_flat_lon, [])
tsne_test_weekday = sum(tsne_test_flat_weekday, [])
tsne_test_output = sum(tsne_test_flat_output, [])
tsne_test_exp_avg = sum(tsne_test_flat_exp_avg,[])

In [44]:
len(tsne_train_flat_lat)

40

### 70 30 split of time bins for Amp and Freq

In [45]:
tsne_train_flat_amp1 = [i[:9169] for i in amp1_allTimeBins]
tsne_train_flat_amp2 = [i[:9169] for i in amp2_allTimeBins]
tsne_train_flat_amp3 = [i[:9169] for i in amp3_allTimeBins]
tsne_train_flat_amp4 = [i[:9169] for i in amp4_allTimeBins]
tsne_train_flat_amp5 = [i[:9169] for i in amp5_allTimeBins]

tsne_train_flat_freq1 = [i[:9169] for i in freq1_allTimeBins]
tsne_train_flat_freq2 = [i[:9169] for i in freq2_allTimeBins]
tsne_train_flat_freq3 = [i[:9169] for i in freq3_allTimeBins]
tsne_train_flat_freq4 = [i[:9169] for i in freq4_allTimeBins]
tsne_train_flat_freq5 = [i[:9169] for i in freq5_allTimeBins]

In [46]:
tsne_test_flat_amp1 = [i[9169:] for i in amp1_allTimeBins]
tsne_test_flat_amp2 = [i[9169:] for i in amp2_allTimeBins]
tsne_test_flat_amp3 = [i[9169:] for i in amp3_allTimeBins]
tsne_test_flat_amp4 = [i[9169:] for i in amp4_allTimeBins]
tsne_test_flat_amp5 = [i[9169:] for i in amp5_allTimeBins]

tsne_test_flat_freq1 = [i[9169:] for i in freq1_allTimeBins]
tsne_test_flat_freq2 = [i[9169:] for i in freq2_allTimeBins]
tsne_test_flat_freq3 = [i[9169:] for i in freq3_allTimeBins]
tsne_test_flat_freq4 = [i[9169:] for i in freq4_allTimeBins]
tsne_test_flat_freq5 = [i[9169:] for i in freq5_allTimeBins]

In [47]:
tsne_train_amp1 = sum(tsne_train_flat_amp1, [])
tsne_train_amp2 = sum(tsne_train_flat_amp2, [])
tsne_train_amp3 = sum(tsne_train_flat_amp3, [])
tsne_train_amp4 = sum(tsne_train_flat_amp4, [])
tsne_train_amp5 = sum(tsne_train_flat_amp5, [])

tsne_train_freq1 = sum(tsne_train_flat_freq1, [])
tsne_train_freq2 = sum(tsne_train_flat_freq2, [])
tsne_train_freq3 = sum(tsne_train_flat_freq3, [])
tsne_train_freq4 = sum(tsne_train_flat_freq4, [])
tsne_train_freq5 = sum(tsne_train_flat_freq5, [])

In [48]:
tsne_test_amp1 = sum(tsne_test_flat_amp1, [])
tsne_test_amp2 = sum(tsne_test_flat_amp2, [])
tsne_test_amp3 = sum(tsne_test_flat_amp3, [])
tsne_test_amp4 = sum(tsne_test_flat_amp4, [])
tsne_test_amp5 = sum(tsne_test_flat_amp5, [])

tsne_test_freq1 = sum(tsne_test_flat_freq1, [])
tsne_test_freq2 = sum(tsne_test_flat_freq2, [])
tsne_test_freq3 = sum(tsne_test_flat_freq3, [])
tsne_test_freq4 = sum(tsne_test_flat_freq4, [])
tsne_test_freq5 = sum(tsne_test_flat_freq5, [])

In [49]:
def standardizeValues(val,isStd):
    if isStd == 'Y':
        z=np.array(val)
        z=z.reshape(-1,1)
        std = StandardScaler().fit_transform(z)
        return std
    else:
        return val

In [50]:
isStd='Y'

# -----------------------------------------------------------------------------------------------------------

# Final Data Massaging To Be Fed Into The Models

In [185]:
# Preparing the data frame for our train data
columns = ['ft_5','ft_4','ft_3','ft_2','ft_1']
df_train = pd.DataFrame(data=train_new_features, columns=columns) 

df_train['lat'] = tsne_train_lat
df_train['lon'] = tsne_train_lon
df_train['weekday'] = tsne_train_weekday
df_train['exp_avg'] = tsne_train_exp_avg

df_train['amp1'] = standardizeValues(tsne_train_amp1,isStd)
df_train['amp2'] = standardizeValues(tsne_train_amp2,isStd)
df_train['amp3'] = standardizeValues(tsne_train_amp3,isStd)
df_train['amp4'] = standardizeValues(tsne_train_amp4,isStd)
df_train['amp5'] = standardizeValues(tsne_train_amp5,isStd)
df_train['freq1'] = standardizeValues(tsne_train_freq1,isStd)
df_train['freq2'] = standardizeValues(tsne_train_freq2,isStd)
df_train['freq3'] = standardizeValues(tsne_train_freq3,isStd)
df_train['freq4'] = standardizeValues(tsne_train_freq4,isStd)
df_train['freq5'] = standardizeValues(tsne_train_freq5,isStd)

print(df_train.shape)

(366760, 20)


In [186]:
# Preparing the data frame for our test data
df_test = pd.DataFrame(data=test_new_features, columns=columns) 
#df_test = pd.DataFrame()
df_test['lat'] = tsne_test_lat
df_test['lon'] = tsne_test_lon
df_test['weekday'] = tsne_test_weekday
df_test['exp_avg'] = tsne_test_exp_avg

df_test['amp1'] = standardizeValues(tsne_test_amp1,isStd)
df_test['amp2'] = standardizeValues(tsne_test_amp2,isStd)
df_test['amp3'] = standardizeValues(tsne_test_amp3,isStd)
df_test['amp4'] = standardizeValues(tsne_test_amp4,isStd)
df_test['amp5'] = standardizeValues(tsne_test_amp5,isStd)
df_test['freq1'] = standardizeValues(tsne_test_freq1,isStd)
df_test['freq2'] = standardizeValues(tsne_test_freq2,isStd)
df_test['freq3'] = standardizeValues(tsne_test_freq3,isStd)
df_test['freq4'] = standardizeValues(tsne_test_freq4,isStd)
df_test['freq5'] = standardizeValues(tsne_test_freq5,isStd)

print(df_test.shape)

(157200, 20)


In [209]:
df_train.head()

,ft_5,ft_4,ft_3,ft_2,ft_1,lat,lon,weekday,exp_avg,amp1,amp2,amp3,amp4,amp5,freq1,freq2,freq3,freq4,freq5
0,0,63,217,189,137,40.776228,-73.982119,4,142,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465
1,63,217,189,137,135,40.776228,-73.982119,4,135,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465
2,217,189,137,135,129,40.776228,-73.982119,4,129,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465
3,189,137,135,129,150,40.776228,-73.982119,4,147,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465
4,137,135,129,150,164,40.776228,-73.982119,4,162,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465


# -----------------------------------------------------------------------------------------------------------

# Task 1:  Applying Regression Models With Foureir Features

<h3> Linear Regression </h3>

In [188]:
from sklearn.linear_model import LinearRegression
lr_reg=LinearRegression().fit(df_train, tsne_train_output)

y_pred = lr_reg.predict(df_test)
lr_test_predictions = [round(value) for value in y_pred]
y_pred = lr_reg.predict(df_train)
lr_train_predictions = [round(value) for value in y_pred]

In [189]:
trMape=(mean_absolute_error(tsne_train_output, lr_train_predictions))/(sum(tsne_train_output)/len(tsne_train_output))

In [190]:
tstMape=(mean_absolute_error(tsne_test_output, lr_test_predictions))/(sum(tsne_test_output)/len(tsne_test_output))

In [191]:
print('Train MAPE : %s . Test MAPE : %s ' %(trMape,tstMape))

Train MAPE : 0.13274292592069048 . Test MAPE : 0.12868880189916243 


<h3> Random Forest </h3>

In [50]:
regr1 = RandomForestRegressor(max_features='sqrt',min_samples_leaf=4,min_samples_split=3,n_estimators=40, n_jobs=-1)
regr1.fit(df_train, tsne_train_output)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=4, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [51]:
# Predicting on test data using our trained random forest model 

# the models regr1 is already hyper parameter tuned
# the parameters that we got above are found using grid search

y_pred = regr1.predict(df_test)
rndf_test_predictions = [round(value) for value in y_pred]
y_pred = regr1.predict(df_train)
rndf_train_predictions = [round(value) for value in y_pred]

In [52]:
fImp=regr1.feature_importances_
col = df_train.columns

In [53]:
index=(np.argsort(fImp))[::-1]

In [54]:
print('Weights: %s ' %(fImp[index]))
print('col: %s ' %(col[index]))

Weights: [0.23256047 0.20371421 0.16139968 0.13044209 0.12451193 0.05210638
 0.0292042  0.02135776 0.01701342 0.01196076 0.00992593 0.00170521
 0.0015913  0.00114796 0.00041639 0.00039203 0.00030693 0.00024335
 0.        ] 
col: Index(['exp_avg', 'ft_3', 'ft_2', 'ft_1', 'ft_5', 'ft_4', 'amp5', 'amp1',
       'amp4', 'amp3', 'amp2', 'lat', 'weekday', 'lon', 'freq4', 'freq5',
       'freq3', 'freq2', 'freq1'],
      dtype='object') 


In [55]:
trMape=(mean_absolute_error(tsne_train_output, rndf_train_predictions))/(sum(tsne_train_output)/len(tsne_train_output))

In [56]:
tstMape=(mean_absolute_error(tsne_test_output, rndf_test_predictions))/(sum(tsne_test_output)/len(tsne_test_output))

In [57]:
print('Train MAPE : %s . Test MAPE : %s ' %(trMape,tstMape))

Train MAPE : 0.0935870325176211 . Test MAPE : 0.1246361658938258 


<h3> XGBOOST </h3>

In [58]:
x_model = xgb.XGBRegressor(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=3,
 gamma=0,
 subsample=0.8,
 reg_alpha=200, reg_lambda=200,
 colsample_bytree=0.8,nthread=4)
x_model.fit(df_train, tsne_train_output)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=3, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
       reg_alpha=200, reg_lambda=200, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [59]:
#predicting with our trained Xg-Boost regressor
# the models x_model is already hyper parameter tuned
# the parameters that we got above are found using grid search

y_pred = x_model.predict(df_test)
xgb_test_predictions = [round(value) for value in y_pred]
y_pred = x_model.predict(df_train)
xgb_train_predictions = [round(value) for value in y_pred]

In [60]:
#feature importances
x_model.get_booster().get_score(importance_type='weight')

{'exp_avg': 751,
 'ft_2': 943,
 'ft_1': 954,
 'ft_3': 743,
 'amp1': 340,
 'ft_4': 634,
 'amp3': 54,
 'amp2': 310,
 'lon': 312,
 'ft_5': 930,
 'lat': 253,
 'amp4': 174,
 'weekday': 303,
 'freq2': 45,
 'freq4': 63,
 'amp5': 27,
 'freq5': 45,
 'freq3': 23}

In [61]:
trMape=(mean_absolute_error(tsne_train_output, xgb_train_predictions))/(sum(tsne_train_output)/len(tsne_train_output))

In [62]:
tstMape=(mean_absolute_error(tsne_test_output, xgb_test_predictions))/(sum(tsne_test_output)/len(tsne_test_output))

In [63]:
print('Train MAPE : %s . Test MAPE : %s ' %(trMape,tstMape))

Train MAPE : 0.12796546250024254 . Test MAPE : 0.12572155174720648 


# -----------------------------------------------------------------------------------------------------------

# Task 2: Hyperparam tuning

<h3> Error Function To Be Used In GridSearchCV and RandomSearchCV </h3>

In [83]:
def myError(y_orig,y_pred):
    error=mean_absolute_error(y_orig, y_pred)/(sum(y_orig)/len(y_orig))
    return error

In [84]:
from sklearn.metrics.scorer import make_scorer
my_scorer = make_scorer(myError, greater_is_better=False)

<h1> Linear Regression </h1>

In [84]:
tuned_parameters = [{'alpha': [0.001,0.01,0.1,1,10] , 'eta0': [10 ** -4,10 ** -5,10 ** -6,10 ** -7,10 ** -8,10 ** -9,10 ** -10]}]
gridSearchModel = GridSearchCV(SGDRegressor(random_state=42,loss='squared_loss',penalty='l2'),tuned_parameters,scoring = my_scorer, cv=5)
gridSearchModel.fit(df_train, tsne_train_output)
results = gridSearchModel.cv_results_

In [86]:
print('Best estimator :{}'.format(gridSearchModel.best_estimator_))
print('\nScore on train data :{}'.format(np.abs(gridSearchModel.score(df_train, tsne_train_output))))
print('\nScore on test data :{}'.format(np.abs(gridSearchModel.score(df_test, tsne_test_output))))

Best estimator :SGDRegressor(alpha=0.01, average=False, epsilon=0.1, eta0=1e-06,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=42, shuffle=True, tol=None, verbose=0,
       warm_start=False)

Score on train data :0.13320983056258268

Score on test data :0.12939730292966478


<h3> Training the model with best hyper params </h3>

In [91]:
bestAlpha=gridSearchModel.best_estimator_.alpha
bestEta=gridSearchModel.best_estimator_.eta0

In [92]:
clf=SGDRegressor(random_state=42,loss='squared_loss',penalty='l2',alpha=bestAlpha,eta0=bestEta)
clf.fit(df_train,tsne_train_output)

y_pred = clf.predict(df_test)
test_predictions = [round(value) for value in y_pred]
y_pred = clf.predict(df_train)
train_predictions = [round(value) for value in y_pred]

train_pred=(mean_absolute_error(tsne_train_output,train_predictions))/(sum(tsne_train_output)/len(tsne_train_output))
test_pred=(mean_absolute_error(tsne_test_output,test_predictions))/(sum(tsne_test_output)/len(tsne_test_output))

print('The best alpha and eta from hyper param tunning is : %s ,  %s ' %(bestAlpha,bestEta))
print('Mape for best alpha and eta is -> Test : %s  Train : %s ' %(test_pred*100,train_pred*100))

The best alpha and eta from hyper param tunning is : 0.01 ,  1e-06 
Mape for best alpha and eta is -> Test : 12.927407115742634  Train : 13.30796878507937 


In [93]:
print(train_predictions[0:10])
print(tsne_train_output[0:10])
print("*" * 50)
print(test_predictions[0:10])
print(tsne_test_output[0:10])

[154.0, 141.0, 129.0, 141.0, 156.0, 152.0, 137.0, 136.0, 141.0, 130.0]
[135, 129, 150, 164, 152, 131, 138, 147, 127, 138]
**************************************************
[99.0, 99.0, 113.0, 124.0, 151.0, 152.0, 139.0, 144.0, 161.0, 154.0]
[101, 120, 131, 164, 154, 133, 148, 172, 153, 162]


<h1> Random Forest </h1>

In [116]:
min_samples_leaf = np.random.uniform(2, 80, 3).astype(int)
min_samples_split = np.random.uniform(2, 10, 3).astype(int)
n_estimators = np.random.uniform(20, 80 ,3).astype(int)

print('min_samples_leaf: %s ' %(min_samples_leaf))
print('min_samples_split: %s ' %(min_samples_split))
print('n_estimators: %s ' %(n_estimators))

tuned_parameters = {'min_samples_leaf':min_samples_leaf, 'min_samples_split':min_samples_split,'n_estimators': n_estimators}
rndSearchRF = RandomizedSearchCV(RandomForestRegressor(max_features='sqrt'),tuned_parameters,scoring = my_scorer, cv=5)
rndSearchRF.fit(df_train, tsne_train_output)
results = rndSearchRF.cv_results_

min_samples_leaf: [53 17 20] 
min_samples_split: [9 8 2] 
n_estimators: [50 79 21] 


In [117]:
print('Best estimator :{}'.format(rndSearchRF.best_estimator_))
print('\nScore on train data :{}'.format(np.abs(rndSearchRF.score(df_train, tsne_train_output))))
print('\nScore on test data :{}'.format(np.abs(rndSearchRF.score(df_test, tsne_test_output))))

Best estimator :RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=17, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

Score on train data :0.11773815194192055

Score on test data :0.12472312005155084


<h3> Training the model with best hyper params </h3>

In [118]:
bestMinSampleLeaf=rndSearchRF.best_estimator_.min_samples_leaf
bestSplit=rndSearchRF.best_estimator_.min_samples_split
bestEstimators=rndSearchRF.best_estimator_.n_estimators

In [119]:
rf=RandomForestRegressor(max_features='sqrt',min_samples_leaf=bestMinSampleLeaf,min_samples_split=bestSplit,n_estimators=bestEstimators)
rf.fit(df_train,tsne_train_output)

y_pred = rf.predict(df_test)
test_predictions = [round(value) for value in y_pred]
y_pred = rf.predict(df_train)
train_predictions = [round(value) for value in y_pred]

train_pred=(mean_absolute_error(tsne_train_output,train_predictions))/(sum(tsne_train_output)/len(tsne_train_output))
test_pred=(mean_absolute_error(tsne_test_output,test_predictions))/(sum(tsne_test_output)/len(tsne_test_output))

print('The best min_samples_leaf from hyper param tunning is : %s ' %(bestMinSampleLeaf))
print('The best bestEstimators from hyper param tunning is : %s ' %(bestEstimators))
print('The best bestSplit from hyper param tunning is : %s ' %(bestSplit))
print('Mape for best hyperparameters is -> Test : %s  Train : %s ' %(test_pred,train_pred))

The best min_samples_leaf from hyper param tunning is : 17 
The best bestEstimators from hyper param tunning is : 50 
The best bestSplit from hyper param tunning is : 2 
Mape for best hyperparameters is -> Test : 0.12459220380061853  Train : 0.11759747386976876 


In [121]:
print(train_predictions[0:10])
print(tsne_train_output[0:10])
print("*" * 50)
print(test_predictions[0:10])
print(tsne_test_output[0:10])

[128.0, 133.0, 133.0, 145.0, 150.0, 143.0, 140.0, 138.0, 140.0, 132.0]
[135, 129, 150, 164, 152, 131, 138, 147, 127, 138]
**************************************************
[104.0, 105.0, 111.0, 125.0, 151.0, 146.0, 138.0, 143.0, 156.0, 151.0]
[101, 120, 131, 164, 154, 133, 148, 172, 153, 162]


<h1>XGBOOST</h1>

In [12]:
min_child_weight = np.random.uniform(2, 8, 2).astype(int)
max_depth = np.random.uniform(2, 8, 2).astype(int)
n_estimators = np.random.uniform(100, 1000 ,2).astype(int)
reg_alpha = np.random.uniform(100, 500 ,2).astype(int)
reg_lambda = np.random.uniform(100, 500 ,2).astype(int)

tuned_parameters = {
    'min_child_weight':min_child_weight,
    'max_depth':max_depth,
    'n_estimators': n_estimators,
    'reg_alpha':reg_alpha,
    'reg_lambda': reg_lambda
}

In [13]:
x_model = xgb.XGBRegressor(
 learning_rate =0.1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,nthread=4)

In [14]:
xgbModelRS = RandomizedSearchCV(x_model,param_distributions=tuned_parameters,scoring = my_scorer, cv=5)
xgbModelRS.fit(df_train,tsne_train_output)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'min_child_weight': array([4, 7]), 'max_depth': array([7, 2]), 'n_estimators': array([877, 795]), 'reg_alpha': array([333, 463]), 'reg_lambda': array([104, 229])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn',
          scoring=make_scorer(myError, greater_is_better=False), verbose=0)

In [15]:
print('Best estimator :{}'.format(xgbModelRS.best_estimator_))
print('\nScore on train data :{}'.format(np.abs(xgbModelRS.score(df_train1, tsne_train_output))))
print('\nScore on test data :{}'.format(np.abs(xgbModelRS.score(df_test1, tsne_test_output))))

Best estimator :XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=7, missing=None, n_estimators=795,
       n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
       reg_alpha=333, reg_lambda=104, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

Score on train data :0.12044026413149364

Score on test data :0.12391724860838926


<h3> Training the model with best hyper params </h3>

In [17]:
bestdepth=xgbModelRS.best_estimator_.max_depth
bestChildWeight=xgbModelRS.best_estimator_.min_child_weight
bestEstimators=xgbModelRS.best_estimator_.n_estimators
bestlambda=xgbModelRS.best_estimator_.reg_lambda
bestalpha=xgbModelRS.best_estimator_.reg_alpha

In [18]:
xgbBest = xgb.XGBRegressor(
 learning_rate =0.1,
 max_depth=bestdepth,
 min_child_weight = bestChildWeight,
 n_estimators = bestEstimators,
 reg_alpha = bestalpha,
 reg_lambda = bestlambda,   
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,nthread=4)

In [20]:
xgbBest.fit(df_train,tsne_train_output)

y_pred = xgbBest.predict(df_test)
test_predictions = [round(value) for value in y_pred]
y_pred = xgbBest.predict(df_train)
train_predictions = [round(value) for value in y_pred]

train_pred=(mean_absolute_error(tsne_train_output,train_predictions))/(sum(tsne_train_output)/len(tsne_train_output))
test_pred=(mean_absolute_error(tsne_test_output,test_predictions))/(sum(tsne_test_output)/len(tsne_test_output))

print('The best bestdepth from hyper param tunning is : %s ' %(bestdepth))
print('The best bestEstimators from hyper param tunning is : %s ' %(bestEstimators))
print('The best bestChildWeight from hyper param tunning is : %s ' %(bestChildWeight))
print('The best bestlambda from hyper param tunning is : %s ' %(bestlambda))
print('The best bestalpha from hyper param tunning is : %s ' %(bestalpha))
print('Mape for best hyperparameters is -> Test : %s  Train : %s ' %(test_pred,train_pred))

The best bestdepth from hyper param tunning is : 7 
The best bestEstimators from hyper param tunning is : 795 
The best bestChildWeight from hyper param tunning is : 7 
The best bestlambda from hyper param tunning is : 104 
The best bestalpha from hyper param tunning is : 333 
Mape for best hyperparameters is -> Test : 0.12384628176261234  Train : 0.1203148256321562 


In [21]:
#feature importances
xgbBest.get_booster().get_score(importance_type='weight')

{'exp_avg': 5797,
 'ft_5': 8570,
 'amp1': 2933,
 'ft_3': 7474,
 'ft_4': 7913,
 'amp2': 3004,
 'amp4': 2481,
 'ft_1': 7321,
 'ft_2': 7732,
 'lat': 2231,
 'lon': 2364,
 'freq4': 1057,
 'amp3': 695,
 'freq5': 668,
 'amp5': 497,
 'freq2': 776,
 'weekday': 3550,
 'freq3': 407}

In [23]:
print(train_predictions[0:10])
print(tsne_train_output[0:10])
print("*" * 50)
print(test_predictions[0:10])
print(tsne_test_output[0:10])

[97.0, 137.0, 132.0, 138.0, 149.0, 149.0, 142.0, 139.0, 138.0, 133.0]
[135, 129, 150, 164, 152, 131, 138, 147, 127, 138]
**************************************************
[105.0, 104.0, 113.0, 121.0, 148.0, 147.0, 142.0, 143.0, 151.0, 150.0]
[101, 120, 131, 164, 154, 133, 148, 172, 153, 162]


# -----------------------------------------------------------------------------------------------------------

# Task 3 : Feature Engineering

### Adding hour of  day  to each row 

This will improve the model as the number of pickups have a dependency on the time of the day

During morning and evening hours the pickups will be more in comaprison to the afternoon

In [48]:
df_train = pd.read_pickle('df_train.pkl')
df_test = pd.read_pickle('df_test.pkl')

In [27]:
with open('tsne_test_output.pkl', 'rb') as f:
    tsne_test_output = pickle.load(f)
with open('tsne_train_output.pkl', 'rb') as f:
    tsne_train_output = pickle.load(f)

In [37]:
df_train.head()

,ft_5,ft_4,ft_3,ft_2,ft_1,lat,lon,weekday,exp_avg,amp1,amp2,amp3,amp4,amp5,freq1,freq2,freq3,freq4,freq5,hours
0,0,63,217,189,137,40.776228,-73.982119,4,150,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465,0
1,63,217,189,137,135,40.776228,-73.982119,4,139,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465,1
2,217,189,137,135,129,40.776228,-73.982119,4,132,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465,1
3,189,137,135,129,150,40.776228,-73.982119,4,144,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465,1
4,137,135,129,150,164,40.776228,-73.982119,4,158,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465,1


In [19]:
oneDay=[]
for i in range(24):
    oneDay.extend([i] *6)  # As there are 144 bins in a day and 6 bins in a hour 
oneRegionTotal = oneDay * 31 + oneDay * 29 + oneDay * 31  # Jan-31 Feb -29 Mar-31
oneRegionTotal = oneRegionTotal[5:]  #Removing the first 5 bin

In [21]:
totalHourClusters=[]
for i in range(40):
    totalHourClusters.append(oneRegionTotal)

In [25]:
tsne_train_hours_flat = [i[:9169] for i in totalHourClusters]
tsne_test_hours_flat = [i[9169:] for i in totalHourClusters]

tsne_train_hours = sum(tsne_train_hours_flat, [])
tsne_test_hours = sum(tsne_test_hours_flat, [])

In [29]:
df_train['hours'] = tsne_train_hours
df_test['hours'] = tsne_test_hours

In [30]:
df_train.head()

,ft_5,ft_4,ft_3,ft_2,ft_1,lat,lon,weekday,exp_avg,amp1,amp2,amp3,amp4,amp5,freq1,freq2,freq3,freq4,freq5,hours
0,0,63,217,189,137,40.776228,-73.982119,4,150,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465,0
1,63,217,189,137,135,40.776228,-73.982119,4,139,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465,1
2,217,189,137,135,129,40.776228,-73.982119,4,132,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465,1
3,189,137,135,129,150,40.776228,-73.982119,4,144,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465,1
4,137,135,129,150,164,40.776228,-73.982119,4,158,0.542451,0.80106,0.80106,0.375128,0.375128,0.0,-1.180805,1.12074,0.900268,-0.950465,1


In [31]:
rf=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=17, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [34]:
rf.fit(df_train,tsne_train_output)

y_pred = rf.predict(df_test)
test_predictions = [round(value) for value in y_pred]
y_pred = rf.predict(df_train)
train_predictions = [round(value) for value in y_pred]

train_pred=(mean_absolute_error(tsne_train_output,train_predictions))/(sum(tsne_train_output)/len(tsne_train_output))
test_pred=(mean_absolute_error(tsne_test_output,test_predictions))/(sum(tsne_test_output)/len(tsne_test_output))

print('Mape for best hyperparameters is -> Test : %s  Train : %s ' %(test_pred,train_pred))

Mape for best hyperparameters is -> Test : 0.119325545034386  Train : 0.11240538098445352 


In [36]:
fImp=rf.feature_importances_
col = df_train.columns
index=(np.argsort(fImp))[::-1]
print('Weights: %s ' %(fImp[index]))
print('col: %s ' %(col[index]))

Weights: [3.14459270e-01 1.99040490e-01 1.71786422e-01 9.81518248e-02
 7.33022740e-02 4.85752397e-02 2.38011340e-02 1.69503480e-02
 1.57608877e-02 1.40856037e-02 1.02099195e-02 9.63251971e-03
 1.48677887e-03 1.38449254e-03 7.53425742e-04 2.73784471e-04
 1.35848483e-04 1.15373473e-04 9.43631081e-05 0.00000000e+00] 
col: Index(['exp_avg', 'ft_1', 'ft_3', 'ft_2', 'ft_5', 'ft_4', 'amp1', 'amp5',
       'amp2', 'amp4', 'amp3', 'hours', 'lon', 'lat', 'weekday', 'freq5',
       'freq4', 'freq3', 'freq2', 'freq1'],
      dtype='object') 
